IMPORTS

In [ ]:
# Python Module Imports
import os
import numpy as np
from time import perf_counter

In [ ]:
# External Imports
import cv2
from keras import Input
from keras.backend import abs as kerasAbs
from keras.models import Sequential, Model, load_model
from keras.layers import Convolution2D, MaxPooling2D, Flatten, Dense, Dropout, Lambda, BatchNormalization, Activation
from matplotlib import pyplot as plt
from mtcnn.mtcnn import MTCNN

In [ ]:
# GUI Imports
from tkinter import Tk
from tkinter.filedialog import askopenfilename
from tkinter.messagebox import showerror

root = Tk()
root.withdraw()

HELPER FUNCTIONS

In [ ]:
def MT_CNN(img, min_face_size=20, scale_factor=0.709):
    """
    Reads an Image and detects the faces in this image using
    a Pre-Trained Multi-task Cascaded Convolutional Neural Networks for Face Detection,
    based on TensorFlow.

    --Parameters--
    img - Matrix of the type CV_8U containing an image where objects are detected.
    min_face_size: Minimum possible object size. Objects smaller than that are ignored.
    scale_factor: Parameter specifying how much the image size is reduced at each image scale.

    """
    model = MTCNN(min_face_size=min_face_size, scale_factor=scale_factor)
    faces = model.detect_faces(img)
    faces = filter(lambda x: x['confidence'] >= 0.90, faces)
    return list(map(lambda x: x['box'], faces))

In [ ]:
def resize_img(img, required_size=(160, 160)):
    """
    Resize the image to the required dimensions.
    """
    return cv2.resize(img, required_size)

In [ ]:
def get_photo():
    """
    Accepts a photo from the user and returns it as a numpy array
    """
    filename = askopenfilename(
        title="Select Test Image",
        filetypes=[("Image Files", "*.jpg;*.jpeg;*.png;")],
        initialdir=os.getcwd(),
        parent=root
    )
    if filename:
        image = cv2.imread(filename)
    else:
        showerror("Error", "No File Selected.")
        raise SystemExit
    return image

In [ ]:
def label_face(image, face, predicted, rect_color=(0, 255, 0), text_color=(0, 0, 0)):
    """
    Labels the face in the image with the predicted label
    image - the image to be labeled
    face - the face to be labeled
    predicted - the label to be applied
    rect_color - the color of the rectangle
    text_color - the color of the text
    """
    x, y, w, h = face
    font_size, font_thiccness = 0.7, 1
    l, b = cv2.getTextSize(
        predicted, cv2.FONT_HERSHEY_SIMPLEX, font_size, font_thiccness)[0]
    cv2.rectangle(image, (x, y), (x+w, y+h), rect_color, 2)
    cv2.rectangle(image, (x-1, y), (x+max(l, w)+1, y-b-10), rect_color, -1)
    cv2.putText(image, predicted, (x, y-6),
                cv2.FONT_HERSHEY_SIMPLEX, font_size, text_color, font_thiccness)
    return image

In [ ]:
def locate_faces(image):
    """
    Detects faces in the image and returns their locations
    image - the image to be analyzed
    """
    faces_loc = MT_CNN(image, min_face_size=60, scale_factor=0.747)
    return faces_loc

In [ ]:
def encode_faces(image, faces_loc):
    """
    Crops the faces from the image and returns their data
    image - the image to be analyzed
    faces_loc - the location of the faces
    """
    faces_data = []
    for (x, y, w, h) in faces_loc:
        face = image[y:y+h, x:x+w]
        face = resize_img(face)
        faces_data.append(face)

    return faces_data

FEW SHOT LEARNING TRAINING AND TESTING FUNCTIONS

In [ ]:
# Constants
# N-way K-shot Learning
N = 6 # number of classes
K = 5 # number of images per class
LIMIT = 8 # number of images to be used for training

In [ ]:
def load_images(path=None, label=0, mode=None):
    """
    Loads images from the given path and returns a list of images.
    path - the path from which to load the images
    label - the label of the images
    mode - the mode of the training.
    """
    if path is None:
        raise ValueError("Path is not defined")

    if mode is None:
        mode = 0

    elif isinstance(mode, int):
        if mode not in [0, 1]:
            raise ValueError("Mode should be either 0 or 1")

    elif isinstance(mode, str):
        if mode not in ["train", "val"]:
            raise ValueError("Mode should be either 'train' or 'val'")
        mode = {"train":0, "val":1}[mode]

    classes = []
    labels = []
    current_label = label
    classes_dict = dict()
    target = 0
    valid_set = [{"Hiranmay", "Neel", "Shilpi"}, {"Shreya", "Srijani", "Richa"}][mode]
    
    for person in os.scandir(path):
        if person.is_dir():
            if person.name not in valid_set:
                continue
            print(f"Loading images from {person.name}")
            classes_dict[person.name] = [current_label, current_label]
            person_images = []
            for img_no, image in enumerate(os.scandir(person.path)):
                if image.is_file():
                    if img_no == LIMIT:
                        break
                    img = cv2.imread(image.path)
                    person_images.append(img)
                    labels.append(target)
                    classes_dict[person.name][1] = current_label
                    current_label += 1
            target += 1
            classes.append(np.stack(person_images))
    labels = np.vstack(labels)
    classes = np.stack(classes)

    return classes, labels, classes_dict

In [ ]:
X_train, y_train, c_train = load_images("dataset/train/labelled data", mode = 0)
X_val, y_val, c_val = load_images("dataset/train/labelled data", mode = 1)

In [ ]:
def test_load_images():
    """
    Tests the load_images function
    """
    mode = 0
    X = [X_train, X_val][mode]
    y = [y_train, y_val][mode]
    targets = ['Hiranmay', 'Neel', 'Richa', 'Shilpi', 'Shreya', 'Srijani']
    fig, axes = plt.subplots(nrows=N//2, ncols=LIMIT, figsize=(28, 28))
    for id, person, label in zip(range(N//2), X, y):
        for idx, img in enumerate(person, start = 0):
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            axes[id, idx].imshow(img)
            axes[id, idx].set_title(f"{targets[id]} {idx+1}")
            axes[id, idx].axis('off')
    plt.show()
test_load_images()

In [ ]:
def load_batch(batch_size, mode = None):
    """
    Loads a batch of images from the given mode.
    batch_size - the size of the batch
    mode - the mode of the training.
    """
    if mode is None:
        mode = 0

    elif isinstance(mode, int):
        if mode not in [0, 1]:
            raise ValueError("Mode should be either 0 or 1")

    elif isinstance(mode, str):
        if mode not in ["train", "val"]:
            raise ValueError("Mode should be either 'train' or 'val'")
        mode = {"train":0, "val":1}[mode]
        
    classes = [X_train, X_val][mode]
    n_classes, n_samples, width, height, channels = classes.shape
    
    random_classes = np.random.choice(n_classes, batch_size)
    
    training_pairs = [np.zeros((batch_size, width, height, channels)) for _ in range(2)]
    training_labels = np.zeros((batch_size,))

    training_labels[batch_size//2:] = 1

    for i in range(batch_size):
        class_id = random_classes[i]
        idx_1 = np.random.randint(0, n_samples)
        training_pairs[0][i,:,:,:] = classes[class_id, idx_1].reshape(width, height, 3)
        
        while True:
            idx_2 = np.random.randint(0, n_samples)
            if idx_2 != idx_1:
                break

        if i >= batch_size // 2:
            class_id2 = class_id 
        else: 
            class_id2 = (class_id + np.random.randint(1, n_classes)) % n_classes
        training_pairs[1][i] = classes[class_id2, idx_2].reshape(width, height, 3)
    
    return training_pairs, training_labels

In [ ]:
def test_load_batch():
    """
    Tests the load_batch function
    """
    batch_size = 6
    pairs, labels = load_batch(batch_size, mode = 0)
    fig, axes = plt.subplots(nrows=batch_size, ncols=2, figsize=(28, 28))
    for id, img1, img2 in zip(range(batch_size), pairs[0], pairs[1]):
        axes[id, 0].imshow(cv2.cvtColor(img1.astype('uint8'), cv2.COLOR_BGR2RGB))
        axes[id, 1].imshow(cv2.cvtColor(img2.astype('uint8'), cv2.COLOR_BGR2RGB))
        axes[id, 1].set_title(f"{labels[id]}")
        axes[id, 0].axis('off')
        axes[id, 1].axis('off')

    plt.show()
test_load_batch()

In [ ]:
def get_siamese_model(input_shape = (160, 160, 3)):
    left_input = Input(input_shape)
    right_input = Input(input_shape)
    
    model = Sequential()
    model.add(Convolution2D(64,(5,5),input_shape=input_shape,activation='relu',kernel_regularizer='l2'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=2,strides=(2,2)))
    model.add(Dropout(0.25))
    
    model.add(Convolution2D(128,(5,5),kernel_regularizer='l2'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=2,strides=(2,2)))
    model.add(Dropout(0.25))
    
    model.add(Convolution2D(128,(5,5),kernel_regularizer='l2'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=2,strides=(2,2)))
    model.add(Dropout(0.25))
    
    model.add(Flatten())
    
    model.add(Dense(512,activation='sigmoid',kernel_regularizer='l2'))
    
    left_emb = model(left_input)
    right_emb = model(right_input)
    
    L1_Layer = Lambda(lambda tensors: kerasAbs(tensors[0] - tensors[1]))
    L1_Dist = L1_Layer([left_emb,right_emb])
    OP = Dense(1,activation='sigmoid',kernel_regularizer='l2')(L1_Dist)
    
    siamese_net = Model(inputs=[left_input,right_input],outputs=OP)
    
    siamese_net.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return siamese_net

In [ ]:
def generate(batch_size, mode = "train"):
    """
    A generator for batches of images.
    """
    while True:
        pairs, targets = load_batch(batch_size, mode)
        yield (pairs, targets)

In [ ]:
def train_model():
    """
    Trains the siamese network.
    """
    model = get_siamese_model()
    history = model.fit(generate(32, mode = "train"),
                        steps_per_epoch=20, 
                        validation_data=generate(32, mode = "val"),
                        validation_steps=10, 
                        epochs=50)
    return model, history


In [ ]:
def save_model(model, name = "model"):
    """
    Saves the model to a file
    """
    model.save(f"{name}.h5")

In [ ]:
def evaluate_model(history):
    """
    Plots the training and validation accuracy and loss.
    """
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.savefig("Model_Accuracy.png")

    plt.clf()

    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.savefig("Model_Loss.png")

In [ ]:
def get_model(train = False):
    """
    Returns a newly trained model if train is set to True, otherwise returns the model from the saved file.
    train - Boolean, whether to train the model or to load it from a file.
    """
    if train:
        model, history = train_model()
        evaluate_model(history)
    else:
        model = load_model("OneShotModel_test.h5")
    return model

In [ ]:
def load_support_set(input_shape=(160,160,3)):
    """
    Loads the support set of images
    """
    support_set = [[None, None] for _ in range(N)]
    path = os.path.join(os.getcwd(), "dataset", "support_set")

    for idx, file in enumerate(os.scandir(path)):
        if file.is_dir():
            continue
        label = file.name.split('.')[0]
        img = cv2.imread(file.path)
        img = resize_img(img, input_shape[:2])

        support_set[idx] = [img, label]
    
    return support_set

In [ ]:
def identify_faces(image, faces_loc, faces_data):
    """
    Identifies the faces in the image and labels them
    image - the image to be labeled
    faces_data - the data of the faces
    faces_loc - the location of the faces
    """
    model = get_model()
    support_set = load_support_set()
    for face_loc, face_data in zip(faces_loc, faces_data):
        predictions = [model.predict([face_data.reshape(1, 160, 160, 3), support_set[i][0].reshape(1, 160, 160, 3)]) for i in range(N)]
        predicted = support_set[np.argmax(predictions)][1]
        # cv2.imshow("Image", face_data)
        # cv2.waitKey(0)
        # cv2.destroyAllWindows()
        # predicted = "Unknown"
        image = label_face(
            image,
            face_loc,
            predicted,
            rect_color=(46, 0, 230),
            text_color=(255, 255, 255)
        )
    return image

In [ ]:
CP1 = perf_counter()
image = get_photo()
CP2 = perf_counter()
faces_loc = locate_faces(image)
CP3 = perf_counter()
faces_data = encode_faces(image, faces_loc)
CP4 = perf_counter()
image = identify_faces(image, faces_loc, faces_data)
CP5 = perf_counter()
print(f'''Time Taken:
Load Image = {CP2-CP1} s
Locate Faces = {CP3-CP2} s
Encode Faces = {(CP4-CP3)*1000} ms
Identify Faces = {(CP5-CP4)*1000} ms''')
cv2.imshow("Image", image)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.imwrite("output.jpg", image)